In [6]:
import os, cv2, shutil
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
def fliplr(src, tvt, aug_name='fliplr'):
    img_dir = 'images'
    img_path = os.path.join(src, tvt, img_dir)
    img_files = os.listdir(img_path)
    img_files.sort()
    dst = f'augmentation/{aug_name}/{tvt}'
    dst_images = os.path.join(src, dst, img_dir)
    label_dir = 'labels'
    label_path = os.path.join(src, tvt, label_dir)
    dst_labels = os.path.join(src, dst, label_dir)
    print(f'source images path = {img_path}')
    print(f'source labels path = {label_path}')
    print(f'destination images path = {dst_images}')
    print(f'destination labels path = {dst_labels}')
    if not os.path.exists(dst_images):
        os.makedirs(dst_images)
    if not os.path.exists(dst_labels):
        os.makedirs(dst_labels)

    for file in tqdm(img_files):
        # print(file)
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            # print(file)
            img = cv2.imread(os.path.join(img_path,file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.flip(img,1)
            plt.imsave(os.path.join(dst_images, aug_name+'_'+file), img)

            ann_path = os.path.join(label_path,f'{file[:-3]}txt')
            ann_aug_path = os.path.join(dst_labels, f'{aug_name}_{file[:-3]}txt')
            if os.path.exists(ann_path):
                if os.path.getsize(ann_path) > 0:
                    with open(ann_path, 'r') as txt:
                        txt = txt.read().splitlines()
                    
                    with open(ann_aug_path, 'w') as dst_file:
                        for line in txt:
                            cl, x, y, w, h = [i for i in line.split(' ')]
                            x = str(f'{1-float(x):.6f}')
                            dst_file.writelines(' '.join([cl, x, y, w, h])+'\n')
                else:
                    shutil.copy(ann_path, ann_aug_path)

In [7]:
src = './datasets4/custom_dataset4/merger_beforeandafter1000'
for tvt in ['train','val','test']:
    fliplr(src=src, tvt=tvt, aug_name='fliplr')

source images path = ./datasets4/custom_dataset4/merger_beforeandafter1000/train/images
source labels path = ./datasets4/custom_dataset4/merger_beforeandafter1000/train/labels
destination images path = ./datasets4/custom_dataset4/merger_beforeandafter1000/augmentation/fliplr/train/images
destination labels path = ./datasets4/custom_dataset4/merger_beforeandafter1000/augmentation/fliplr/train/labels


  0%|          | 0/14761 [00:00<?, ?it/s]

source images path = ./datasets4/custom_dataset4/merger_beforeandafter1000/val/images
source labels path = ./datasets4/custom_dataset4/merger_beforeandafter1000/val/labels
destination images path = ./datasets4/custom_dataset4/merger_beforeandafter1000/augmentation/fliplr/val/images
destination labels path = ./datasets4/custom_dataset4/merger_beforeandafter1000/augmentation/fliplr/val/labels


  0%|          | 0/3931 [00:00<?, ?it/s]

source images path = ./datasets4/custom_dataset4/merger_beforeandafter1000/test/images
source labels path = ./datasets4/custom_dataset4/merger_beforeandafter1000/test/labels
destination images path = ./datasets4/custom_dataset4/merger_beforeandafter1000/augmentation/fliplr/test/images
destination labels path = ./datasets4/custom_dataset4/merger_beforeandafter1000/augmentation/fliplr/test/labels


  0%|          | 0/1965 [00:00<?, ?it/s]

In [11]:
import cv2, os, shutil
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
def generate_random_lines(imshape,slant,drop_length):
    drops=[]    
    for i in range(np.random.randint(100,1500)): ## If You want heavy rain, try increasing this        
        x= np.random.randint(0,imshape[1]-slant)        
        y= np.random.randint(0,imshape[0]-drop_length)        
        drops.append((x,y))    
    return drops            

def add_rain(image):        
    imshape = image.shape    
    slant_extreme=np.random.randint(2,20) # default 10
    slant= np.random.randint(-slant_extreme,slant_extreme)     
    drop_length=np.random.randint(2,20) # default 20
    drop_width=1 # default 2
    drop_color=(200,200,200) ## a shade of gray    
    # print(imshape, slant, drop_length)

    blur = np.random.randint(2,5)
    image= cv2.blur(image,(blur,blur)) ## rainy view are blurry        
    rain_drops= generate_random_lines(imshape,slant,drop_length)        
    for rain_drop in rain_drops:        
        cv2.line(img=image,pt1=(rain_drop[0],rain_drop[1]),pt2=(rain_drop[0]+slant,rain_drop[1]+drop_length),color=drop_color,thickness=drop_width)    
        brightness_coefficient = np.random.randint(7,10)/10 ## rainy days are usually shady    defalut 0.7 
        image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS    
        image_HLS[:,:,1] = image_HLS[:,:,1]*brightness_coefficient ## scale pixel values down for channel 1(Lightness)    
        image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB    
    return image_RGB

def rain(src=src, tvt=tvt, aug_name='rain'):
    img_dir = 'images'
    img_path = os.path.join(src, tvt, img_dir)
    img_files = os.listdir(img_path)
    img_files.sort()
    dst = f'augmentation/{aug_name}/{tvt}'
    dst_images = os.path.join(src, dst, img_dir)
    label_dir = 'labels'
    label_path = os.path.join(src, tvt, label_dir)
    dst_labels = os.path.join(src, dst, label_dir)
    print(f'source images path = {img_path}')
    print(f'source labels path = {label_path}')
    print(f'destination images path = {dst_images}')
    print(f'destination labels path = {dst_labels}')
    if not os.path.exists(dst_images):
        os.makedirs(dst_images)
    if not os.path.exists(dst_labels):
        os.makedirs(dst_labels)

    for file in tqdm(img_files):
        # print(file)
        img = cv2.imread(os.path.join(img_path,file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = add_rain(img)
        plt.imsave(os.path.join(dst_images, aug_name+'_'+file), img)
        ann_path = os.path.join(label_path,f'{file[:-3]}txt')
        if os.path.exists(ann_path):
            ann_aug_path = os.path.join(dst_labels, f'{aug_name}_{file[:-3]}txt')
            shutil.copy(ann_path, ann_aug_path)    

In [ ]:
src = './datasets4/custom_dataset4/merger_beforeandafter1000'
for tvt in ['train','val','test']:
    rain(src=src, tvt=tvt, aug_name='rain')

# Merge funcion

In [13]:
import os
import cv2
import shutil
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np

class Augmentation:
    def __init__(self, src, tvt):
        self.src = src
        self.tvt = tvt
        self.img_dir = 'images'
        self.label_dir = 'labels'

    def _get_paths(self, aug_name):
        img_path = os.path.join(self.src, self.tvt, self.img_dir)
        img_files = sorted(os.listdir(img_path))
        dst = f'augmentation/{aug_name}/{self.tvt}'
        dst_images = os.path.join(self.src, dst, self.img_dir)
        label_path = os.path.join(self.src, self.tvt, self.label_dir)
        dst_labels = os.path.join(self.src, dst, self.label_dir)
        return img_path, img_files, dst_images, label_path, dst_labels

    def _create_dirs(self, dst_images, dst_labels):
        os.makedirs(dst_images, exist_ok=True)
        os.makedirs(dst_labels, exist_ok=True)

    def _process_image(self, img_path, file, aug_function):
        img = cv2.imread(os.path.join(img_path, file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = aug_function(img)
        return img

    def _save_image(self, dst_images, aug_name, file, img):
        plt.imsave(os.path.join(dst_images, aug_name + '_' + file), img)

    def _copy_or_update_labels(self, label_path, dst_labels, aug_name, file, update_function=None):
        ann_path = os.path.join(label_path, f'{file[:-3]}txt')
        if os.path.exists(ann_path):
            ann_aug_path = os.path.join(dst_labels, f'{aug_name}_{file[:-3]}txt')
            if os.path.exists(ann_path):
                if os.path.getsize(ann_path) > 0 and update_function:
                    with open(ann_path, 'r') as txt, open(ann_aug_path, 'w') as dst_file:
                        for line in txt:
                            cl, x, y, w, h = [i for i in line.split(' ')]
                            updated_line = update_function(cl, x, y, w, h)
                            dst_file.writelines(updated_line + '\n')
                else:
                    shutil.copy(ann_path, ann_aug_path)

    def fliplr(self, aug_name='fliplr'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def update_labels(cl, x, y, w, h):
            x = str(f'{1 - float(x):.6f}')
            return ' '.join([cl, x, y, w, h])

        for file in tqdm(img_files):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                self._save_image(dst_images, aug_name, file, img)
                self._copy_or_update_labels(label_path, dst_labels, aug_name, file, update_labels)

    def rain(self, aug_name='rain'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def generate_random_lines(imshape, slant, drop_length):
            drops = []
            for i in range(np.random.randint(100, 1500)):
                x = np.random.randint(0, imshape[1] - slant)
                y = np.random.randint(0, imshape[0] - drop_length)
                drops.append((x, y))
            return drops

        def add_rain(image):
            imshape = image.shape
            slant_extreme = np.random.randint(2, 20) 
            slant = np.random.randint(-slant_extreme, slant_extreme)
            drop_length = np.random.randint(2, 20) 
            drop_width = 1  
            drop_color = (200, 200, 200)  
            blur = np.random.randint(2, 5)
            image = cv2.blur(image, (blur, blur)) 
            rain_drops = generate_random_lines(imshape, slant, drop_length)
            for rain_drop in rain_drops:
                cv2.line(img=image, pt1=(rain_drop[0], rain_drop[1]),
                         pt2=(rain_drop[0] + slant, rain_drop[1] + drop_length), color=drop_color, thickness=drop_width)
            brightness_coefficient = np.random.randint(7, 10) / 10 
            image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS) 
            image_HLS[:, :, 1] = image_HLS[:, :, 1] * brightness_coefficient 
            image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB) 
            return image_RGB

        for file in tqdm(img_files):
            img = self._process_image(img_path, file, add_rain)
            self._save_image(dst_images, aug_name, file, img)
            self._copy_or_update_labels(label_path, dst_labels, aug_name, file)

In [15]:
# Create an instance of the Augmentation class
# augmentation = Augmentation(src, tvt)

# # Apply horizontal flip augmentation
# augmentation.fliplr()

# # Apply rain augmentation
# augmentation.rain()

src = './datasets4/custom_dataset4/merger_beforeandafter1000'
for tvt in ['train','val','test']:
    augmentation = Augmentation(src, tvt)
    augmentation.rain()

  0%|          | 0/14761 [00:00<?, ?it/s]

  0%|          | 0/3931 [00:00<?, ?it/s]

  0%|          | 0/1965 [00:00<?, ?it/s]

# Mix rain, fliplr, both in random

In [1]:
import os
import cv2
import shutil
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import random

class Augmentation:
    def __init__(self, src, tvt):
        self.src = src
        self.tvt = tvt
        self.img_dir = 'images'
        self.label_dir = 'labels'

    def _get_paths(self, aug_name):
        img_path = os.path.join(self.src, self.tvt, self.img_dir)
        img_files = sorted(os.listdir(img_path))
        dst = f'augmentation/{aug_name}/{self.tvt}'
        dst_images = os.path.join(self.src, dst, self.img_dir)
        label_path = os.path.join(self.src, self.tvt, self.label_dir)
        dst_labels = os.path.join(self.src, dst, self.label_dir)
        return img_path, img_files, dst_images, label_path, dst_labels

    def _create_dirs(self, dst_images, dst_labels):
        os.makedirs(dst_images, exist_ok=True)
        os.makedirs(dst_labels, exist_ok=True)

    def _process_image(self, img_path, file, aug_function):
        img = cv2.imread(os.path.join(img_path, file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = aug_function(img)
        return img

    def _save_image(self, dst_images, aug_name, file, img):
        plt.imsave(os.path.join(dst_images, aug_name + '_' + file), img)

    def _copy_or_update_labels(self, label_path, dst_labels, aug_name, file, update_function=None):
        ann_path = os.path.join(label_path, f'{file[:-3]}txt')
        if os.path.exists(ann_path):
            ann_aug_path = os.path.join(dst_labels, f'{aug_name}_{file[:-3]}txt')
            if os.path.exists(ann_path):
                if os.path.getsize(ann_path) > 0 and update_function:
                    with open(ann_path, 'r') as txt, open(ann_aug_path, 'w') as dst_file:
                        for line in txt:
                            cl, x, y, w, h = [i for i in line.split(' ')]
                            updated_line = update_function(cl, x, y, w, h)
                            dst_file.writelines(updated_line + '\n')
                else:
                    shutil.copy(ann_path, ann_aug_path)

    def fliplr(self, aug_name='fliplr'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def update_labels(cl, x, y, w, h):
            x = str(f'{1 - float(x):.6f}')
            return ' '.join([cl, x, y, w, h])

        for file in tqdm(img_files):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                self._save_image(dst_images, aug_name, file, img)
                self._copy_or_update_labels(label_path, dst_labels, aug_name, file, update_labels)

    def rain(self, aug_name='rain'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def generate_random_lines(imshape, slant, drop_length):
            drops = []
            for i in range(np.random.randint(100, 1500)):
                x = np.random.randint(0, imshape[1] - slant)
                y = np.random.randint(0, imshape[0] - drop_length)
                drops.append((x, y))
            return drops

        def add_rain(image):
            imshape = image.shape
            slant_extreme = np.random.randint(2, 20) 
            slant = np.random.randint(-slant_extreme, slant_extreme)
            drop_length = np.random.randint(2, 20) 
            drop_width = 1  
            drop_color = (200, 200, 200)  
            blur = np.random.randint(2, 5)
            image = cv2.blur(image, (blur, blur)) 
            rain_drops = generate_random_lines(imshape, slant, drop_length)
            for rain_drop in rain_drops:
                cv2.line(img=image, pt1=(rain_drop[0], rain_drop[1]),
                         pt2=(rain_drop[0] + slant, rain_drop[1] + drop_length), color=drop_color, thickness=drop_width)
            brightness_coefficient = np.random.randint(7, 10) / 10 
            image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS) 
            image_HLS[:, :, 1] = image_HLS[:, :, 1] * brightness_coefficient 
            image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB) 
            return image_RGB

        for file in tqdm(img_files):
            img = self._process_image(img_path, file, add_rain)
            self._save_image(dst_images, aug_name, file, img)
            self._copy_or_update_labels(label_path, dst_labels, aug_name, file)

    def random_augment(self, aug_name='random'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def update_labels_fliplr(cl, x, y, w, h):
            x = str(f'{1 - float(x):.6f}')
            return ' '.join([cl, x, y, w, h])

        def generate_random_lines(imshape, slant, drop_length):
            drops = []
            for i in range(np.random.randint(100, 1500)):
                x = np.random.randint(0, imshape[1] - slant)
                y = np.random.randint(0, imshape[0] - drop_length)
                drops.append((x, y))
            return drops

        def add_rain(image):
            imshape = image.shape
            slant_extreme = np.random.randint(2, 20) 
            slant = np.random.randint(-slant_extreme, slant_extreme)
            drop_length = np.random.randint(2, 20) 
            drop_width = 1  
            drop_color = (200, 200, 200)  
            blur = np.random.randint(2, 5)
            image = cv2.blur(image, (blur, blur)) 
            rain_drops = generate_random_lines(imshape, slant, drop_length)
            for rain_drop in rain_drops:
                cv2.line(img=image, pt1=(rain_drop[0], rain_drop[1]),
                         pt2=(rain_drop[0] + slant, rain_drop[1] + drop_length), color=drop_color, thickness=drop_width)
            brightness_coefficient = np.random.randint(7, 10) / 10 
            image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS) 
            image_HLS[:, :, 1] = image_HLS[:, :, 1] * brightness_coefficient 
            image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB) 
            return image_RGB

        for file in tqdm(img_files):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                choice = random.choice(['fliplr', 'rain', 'both'])
                if choice == 'fliplr':
                    img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                    self._save_image(dst_images, aug_name, 'fliplr_' + file, img)
                    self._copy_or_update_labels(label_path, dst_labels, 'fliplr_' + file[:-4], file, update_labels_fliplr)
                elif choice == 'rain':
                    img = self._process_image(img_path, file, add_rain)
                    self._save_image(dst_images, aug_name, 'rain_' + file, img)
                    self._copy_or_update_labels(label_path, dst_labels, 'rain_' + file[:-4], file)
                else:  # 'both'
                    img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                    img = add_rain(img)
                    self._save_image(dst_images, aug_name, 'both_' + file, img)
                    self._copy_or_update_labels(label_path, dst_labels, 'both_' + file[:-4], file, update_labels_fliplr) 

In [3]:
src = './datasets4/custom_dataset4/merger_beforeandafter1000'
for tvt in ['train','val','test']:
    augmentation = Augmentation(src, tvt)
    augmentation.random_augment()

  0%|          | 0/14761 [00:00<?, ?it/s]

  0%|          | 0/3931 [00:00<?, ?it/s]

  0%|          | 0/1965 [00:00<?, ?it/s]

# Shorten again

In [5]:
import os
import cv2
import shutil
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
import random

class Augmentation:
    def __init__(self, src, tvt):
        self.src = src
        self.tvt = tvt
        self.img_dir = 'images'
        self.label_dir = 'labels'

    def _get_paths(self, aug_name):
        img_path = os.path.join(self.src, self.tvt, self.img_dir)
        img_files = sorted(os.listdir(img_path))
        dst = f'augmentation/{aug_name}/{self.tvt}'
        dst_images = os.path.join(self.src, dst, self.img_dir)
        label_path = os.path.join(self.src, self.tvt, self.label_dir)
        dst_labels = os.path.join(self.src, dst, self.label_dir)
        return img_path, img_files, dst_images, label_path, dst_labels

    def _create_dirs(self, dst_images, dst_labels):
        os.makedirs(dst_images, exist_ok=True)
        os.makedirs(dst_labels, exist_ok=True)

    def _process_image(self, img_path, file, aug_function):
        img = cv2.imread(os.path.join(img_path, file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = aug_function(img)
        return img

    def _save_image(self, dst_images, aug_name, file, img):
        plt.imsave(os.path.join(dst_images, aug_name + '_' + file), img)

    def _copy_or_update_labels(self, label_path, dst_labels, aug_name, file, update_function=None):
        ann_path = os.path.join(label_path, f'{file[:-3]}txt')
        if os.path.exists(ann_path):
            ann_aug_path = os.path.join(dst_labels, f'{aug_name}_{file[:-3]}txt')
            if os.path.exists(ann_path):
                if os.path.getsize(ann_path) > 0 and update_function:
                    with open(ann_path, 'r') as txt, open(ann_aug_path, 'w') as dst_file:
                        for line in txt:
                            cl, x, y, w, h = [i for i in line.split(' ')]
                            updated_line = update_function(cl, x, y, w, h)
                            dst_file.writelines(updated_line + '\n')
                else:
                    shutil.copy(ann_path, ann_aug_path)

    def generate_random_lines(self, imshape, slant, drop_length):
        drops = []
        for i in range(np.random.randint(100, 1500)):
            x = np.random.randint(0, imshape[1] - slant)
            y = np.random.randint(0, imshape[0] - drop_length)
            drops.append((x, y))
        return drops
    def add_rain(self, image):
        imshape = image.shape
        slant_extreme = np.random.randint(2, 20) 
        slant = np.random.randint(-slant_extreme, slant_extreme)
        drop_length = np.random.randint(2, 20) 
        drop_width = 1  
        drop_color = (200, 200, 200)  
        blur = np.random.randint(2, 5)
        image = cv2.blur(image, (blur, blur)) 
        rain_drops = self.generate_random_lines(imshape, slant, drop_length)
        for rain_drop in rain_drops:
            cv2.line(img=image, pt1=(rain_drop[0], rain_drop[1]),
                     pt2=(rain_drop[0] + slant, rain_drop[1] + drop_length), color=drop_color, thickness=drop_width)
        brightness_coefficient = np.random.randint(7, 10) / 10 
        image_HLS = cv2.cvtColor(image, cv2.COLOR_RGB2HLS) 
        image_HLS[:, :, 1] = image_HLS[:, :, 1] * brightness_coefficient 
        image_RGB = cv2.cvtColor(image_HLS, cv2.COLOR_HLS2RGB) 
        return image_RGB


    def fliplr(self, aug_name='fliplr'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def update_labels(cl, x, y, w, h):
            x = str(f'{1 - float(x):.6f}')
            return ' '.join([cl, x, y, w, h])

        for file in tqdm(img_files):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                self._save_image(dst_images, aug_name, file, img)
                self._copy_or_update_labels(label_path, dst_labels, aug_name, file, update_labels)

    def rain(self, aug_name='rain'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        for file in tqdm(img_files):
            img = self._process_image(img_path, file, self.add_rain)
            self._save_image(dst_images, aug_name, file, img)
            self._copy_or_update_labels(label_path, dst_labels, aug_name, file)

    def random_augment(self, aug_name='random'):
        img_path, img_files, dst_images, label_path, dst_labels = self._get_paths(aug_name)
        self._create_dirs(dst_images, dst_labels)

        def update_labels_fliplr(cl, x, y, w, h):
            x = str(f'{1 - float(x):.6f}')
            return ' '.join([cl, x, y, w, h])

        for file in tqdm(img_files):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                choice = random.choice(['fliplr', 'rain', 'both'])
                if choice == 'fliplr':
                    img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                    self._save_image(dst_images, aug_name, 'fliplr_' + file, img)
                    self._copy_or_update_labels(label_path, dst_labels, 'fliplr_' + file[:-4], file, update_labels_fliplr)
                elif choice == 'rain':
                    img = self._process_image(img_path, file, self.add_rain)
                    self._save_image(dst_images, aug_name, 'rain_' + file, img)
                    self._copy_or_update_labels(label_path, dst_labels, 'rain_' + file[:-4], file)
                else:  # 'both'
                    img = self._process_image(img_path, file, lambda img: cv2.flip(img, 1))
                    img = self.add_rain(img)
                    self._save_image(dst_images, aug_name, 'both_' + file, img)
                    self._copy_or_update_labels(label_path, dst_labels, 'both_' + file[:-4], file, update_labels_fliplr) 

In [6]:
src = './datasets4/custom_dataset4/merger_beforeandafter1000'
for tvt in ['train','val','test']:
    augmentation = Augmentation(src, tvt)
    augmentation.random_augment()

  0%|          | 0/14761 [00:00<?, ?it/s]